# 卷积神经网络
### 理论依据
卷积层的依据来源于图像的两个直观的性质
- 平移不变性：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应，即为“平移不变性”。
- 局部性：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系，这就是“局部性”原则。

图像的平移不变性使我们以相同的方式处理局部图像，而不在乎它的位置。

局部性意味着计算相应的隐藏表示只需一小部分局部图像像素。
$$
[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}.
$$
卷积层学习的参数是卷积核，优势是将参数数量从数十亿（线性层）降低到了几百（卷积核一般都很小），代价是假定了特征的平移不变和每层只包含局部信息

### 多通道卷积
对于输入通道c_in，每个通道一个卷积核，各个通道完成卷积计算之后sum()

对于输出通道c_out，每个通道一组卷积核，组大小c_in，简单说就是c_out个c_in分别输出作为不同通道。

所以一个卷基层的总共卷积核数量为c_in*c_out

In [ ]:
# conv by hand
import torch
from torch import nn

def conv2d(X,K:torch.Tensor)->torch.Tensor:
    h,w = K.shape
    res = torch.zeros(X.shape[0]-h+1,X.shape[1]-w+1)
    for i in range(res.shape[0]):
        for j in range(res.shape[1]):
            res[i,j] = (X[i:i+h,j:j+w]*K).sum()
    return res

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return conv2d(x,self.weight) + self.bias

# 设置数据
X = torch.ones(6,8)
X[:,2:6] = 0
K = torch.tensor([[1,-1]])
Y = conv2d(X,K)
print(Y)

# 训练
net = Conv2D((1,2))
for epoch in range(10):
    Y_hat = net(X)
    l = (Y - Y_hat)**2
    net.zero_grad()
    l.sum().backward()
    net.weight.data[:] -= 0.03 * net.weight.grad
    print(f'epoch{epoch+1} loss:{l.sum():.3f}')

print(net.weight.data)

# conv by nn
# 注意前两个参数分别表示输入通道数量和输出通道数量，因此总卷积和数量是c_in*c_out
net = nn.Conv2d(1,1,(1,2),1,0,bias=False)

### 汇聚层
卷积神经网络中，对于某一层的任意元素x，其感受野（receptive field）是指在前向传播期间可能影响x计算的所有元素（来自所有先前层）。可以理解的是，如果图像很大，卷积核很小，那么前几层的感受野应该是很小的。

通常当我们处理图像时，我们希望逐渐降低隐藏表示的空间分辨率、聚集信息，这样随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大。

汇聚层（pooling），它具有双重目的：降低卷积层对位置的敏感性，同时降低对空间降采样表示的敏感性。

pooling层的形状和滑动方式类似卷积核，但是没有任何参数，它只计算最大或平均。

在处理多通道输入数据时，汇聚层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总。 这意味着汇聚层的输出通道数与输入通道数相同。

### LeNet
LeNet，它是最早发布的卷积神经网络之一，因其在计算机视觉任务中的高效性能而受到广泛关注。 这个模型是由AT&T贝尔实验室的研究员Yann LeCun在1989年提出的（并以其命名）


In [ ]:
# LeNet
import torch
from torch import nn

net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))